<a href="https://colab.research.google.com/github/fastdatascience/clinical_trial_risk/blob/fixes_nov_2022/train/ctgov/TrainNumArmsClassifier_19_Spacy_Textcat_FilteredBySentence_in_notebook_augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

#df_annotations = pd.read_csv("../../data/ctgov/annotations/all_annotations.csv")


# Get data

In [ ]:
# with open("../../data/ctgov/protocols.pkl.gz", "rb") as f:
#     file_to_pages = pkl.load(f)

In [ ]:
# import spacy
# nlp = spacy.blank("en")

In [ ]:
# texts = []

# for i in range(len(df_annotations)):
#     file_name = df_annotations.file.iloc[i]
    
#     pages= file_to_pages[file_name]
    
#     text = ""
#     for page in pages:
#         doc = nlp(page)
        
#         is_include = [False] * len(doc)
#         for tok in doc:
#             if tok.text.lower() in {"arm", "arms", "armed", "cohort", "cohorts", "group", "groups"}:
#                 for token_index in range(tok.i - 20, tok.i + 20):
#                     if token_index >= 0 and token_index < len(doc):
#                         is_include[token_index] = True
        
#         for token_index in range(len(doc)):
#             if is_include[token_index]:
#                 text += doc[token_index].text + doc[token_index].whitespace_
                
#     if text == "":
#         print ("nothing found", file_name, df_annotations.num_arms.iloc[i])
#     else:
#         print ("found", file_name, df_annotations.num_arms.iloc[i])

#     texts.append(text)
# df_annotations["text"] = texts

In [ ]:
# df_annotations.to_pickle("filtered_for_arm.pkl")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_annotations = pd.read_csv("/content/drive/MyDrive/data/filtered_for_phase_arms_subjects.csv.bz2")

In [ ]:
# del file_to_pages

In [ ]:
df_annotations_num_arms_known = df_annotations[~df_annotations.num_arms.isna() & ~df_annotations.text.isna()]

In [ ]:
#df_annotations_num_arms_known = df_annotations_num_arms_known.sample(1000)

In [ ]:
def get_num_arms_clean(num):
    if num >= 5:
        num = 5
    return num
df_annotations_num_arms_known["num_arms_clean"] = df_annotations_num_arms_known["num_arms"].apply(get_num_arms_clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df_train = df_annotations_num_arms_known[df_annotations_num_arms_known.train_val == "train"]

In [ ]:
df_train.num_arms_clean.value_counts()

2.0    4173
1.0    2474
3.0    1175
4.0     739
5.0     712
Name: num_arms_clean, dtype: int64

In [ ]:
# TRAINING_DATA = [
#     ["My little kitty is so special", {"KAT0": True}],
#     ["Dude, Totally, Yeah, Video Games", {"KAT1": True}],
#     ["Should I pay $1,000 for the iPhone X?", {"KAT1": True}],
#     ["The iPhone 8 reviews are here", {"KAT1": True}],
#     ["Noa is a great cat name.", {"KAT0": True}],
#     ["We got a new kitten!", {"KAT0": True}]
# ]

TRAINING_DATA = []
for idx in range(len(df_train)):
    cats = {}
    for a in range(1, 6):
        cats[str(a)] = 0
    cats[str(int(df_train.num_arms_clean.iloc[idx]))] = 1
    text = df_train.text.iloc[idx]
    if len(text) > 1000000:
        text = text[:1000000]
    
    aug = 1
    if df_train.num_arms_clean.iloc[idx] == 3:
      aug = 2
    elif df_train.num_arms_clean.iloc[idx] in [4,5]:
      aug = 3
    for i in range(aug):
      TRAINING_DATA.append([text , cats])

In [46]:
import spacy
# Add imports for example, as well as textcat config...
from spacy.training import Example
from spacy.pipeline.textcat import single_label_bow_config, single_label_default_config
from thinc.api import Config
import random

# labels should be one-hot encoded



# bow
# config = Config().from_str(single_label_bow_config)

# textensemble with attention
config = Config().from_str(single_label_default_config)

nlp = spacy.blank("en")
# now uses `add_pipe` instead
category = nlp.add_pipe("textcat", last=True, config=config)
for a in range(1, 6):
    category.add_label(str(a))


# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(100):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=4):
        texts = [nlp.make_doc(text) for text, entities in batch]
        annotations = [{"cats": entities} for text, entities in batch]

        # uses an example object rather than text/annotation tuple
        examples = [Example.from_dict(doc, annotation) for doc, annotation in zip(
            texts, annotations
        )]
        nlp.update(examples, losses=losses)
    if itn % 20 == 0:
        print(losses)
        break

KeyboardInterrupt: ignored

In [48]:
itn

0

In [47]:
1

1

In [49]:
df_annotations_num_arms_known = df_annotations[~df_annotations.num_arms.isna()]

In [50]:
df_annotations_num_arms_known["num_arms_clean"] = df_annotations_num_arms_known["num_arms"].apply(get_num_arms_clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
df_val = df_annotations_num_arms_known[df_annotations_num_arms_known.train_val == "val"]

In [ ]:
predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
df_val["y_pred"] = [int(max(p, key=p.get)) for p in predictions]

In [ ]:
y_pred_proba = np.asarray([np.asarray([p.get(str(i)) for i in range(1,6)]) for p in predictions])

In [ ]:
num_arms_ground_truths = df_val.num_arms_clean
pred_num_arms = df_val["y_pred"]

In [ ]:
acc = accuracy_score(num_arms_ground_truths, pred_num_arms)
print (f"Num arms accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(num_arms_ground_truths, pred_num_arms)
plt.xticks(rotation=90)
;

In [ ]:
# nlp.to_disk("/content/drive/MyDrive/data/textcat_numarms_19.spacy")


In [ ]:
pred_num_arms_proba= y_pred_proba

In [ ]:
all_aucs = []
from sklearn.metrics import roc_auc_score
for i in range(1,5+1):
  roc_auc = roc_auc_score(num_arms_ground_truths == i, pred_num_arms_proba[:, i - 1])
  print (f"Num arms ROC AUC {roc_auc} for {i} arm(s)")
  all_aucs.append(roc_auc)
print ("Average", np.mean(all_aucs))

In [ ]:
plt.figure()
lw = 2
    
for i in range(1,6):
    fpr, tpr, _ = roc_curve(num_arms_ground_truths == i, pred_num_arms_proba[:, i - 1])
    roc_auc = auc(fpr, tpr)
    plt.plot(
        fpr,
        tpr,
        # color="darkorange",
        lw=lw,
        label=f"ROC curve {i} arms (area = {roc_auc:0.2f})"
    )
    plt.plot([0, 1], [0, 1], color="gray", alpha=0.5, lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()